# Pergunta 3 - IDA*
## Avaliação Contínua II - IIA 23/24

<img src="Imagens\chaosAsteriskus.gif" alt="Drawing" style="width: 200px;"/>

## Introdução
O Iterative Deepening `A*` é uma mistura entre o Aprofundamento Progressivo e o `A*`. Tem as vantagens e desvantagens do Aprofundamento Progressivo: a fronteira da árvore de procura cresce pouco, tendo uma complexidade espacial que é é linear, mas devido à sua progressividade irá visitar repetidamente os mesmos nós.
Queremos verificar se este algoritmo é adequado para o problema MedoTotal, comparando-o com o `A*`.

## Objectivo
O objectivo é desenvolver a função `ida_star_graph_search_count` que dado uma instância da classe `Problem` como input, e a função $f(n)=custo(n) + heurística(n)$ executa uma procura de acordo com o `IDA*` na sua versão em grafo.

Notem que em cada passo, é executada uma procura em Profundidade-primeiro em grafo limitada a nós com $f(n)<limiar$, e em que o limiar começa com o valor heurístico do estado inicial e vai progressivamente aumentando se a procura em Profundidade-primeiro limitada não encontrar a solução dentro desse valor limiar. Entre cada passo, o novo limiar será o menor valor $f(n)$ que ultrapassa o limiar actual. O algoritmo devolve a primeira solução que encontra se tiver um valor igual ou menor do que o valor limiar. Essa solução será óptima se a heurística for admissível no caso do IDA* em árvore ou se for consistente no caso da versão em grafo.

Pretendemos a versão em grafo e que respeita a ordem das acções. Devem ter atenção que a procura em Profundidade-primeiro standard usa uma pilha e que naturalmente inverte essa ordem. Na versão em grafo, em cada passo com procura em Profundidade-primeiro nunca se revisitam os estados.

A Profundidade primeiro deve também fazer o teste de satisfação do objectivo quando se faz o pop da fronteira.

O output da função pedida devolverá um tuplo `(node,visitados)` em que `node` é o nó final da procura se o encontrar ou `None` e visitados indica o resultado de contar os estados visitados ao longo dos vários passos de todo o processo de procura. 

Queremos que juntem aos argumentos da função um novo input booleano, a `False` por omissão, que se queremos ou não a versão pedagógica. Na versão pedagógica podemos seguir o rasto do processo de procura desde o estado inicial até encontrar a solução, passo a passo. Ilustraremos o rasto da procura com um exemplo.

Para simplificar iremos usar grafos abstractos para ilustrar a versão pedagógica. Nos testes usaremos grafos abstractos e também instâncias do problema Medo Total.

## A classe GrafoAbstractoHs
Vamos mostrar a implementação da classe abstracta `GrafoAbstracto` que está no ficheiro anexo `GrafoAbstracto.py`. Esta classe tem o grafo seguinte por omissão. 

<img src="Imagens\GrafoDoisFinais.PNG" alt="Drawing" style="width: 450px;"/>

com o valor da função heurística inscrita ao lado dos estados.

``` python
class ProblemaGrafoHs(Problem) :
    grafoDuplo= {'I':{'A':3,'B':3},
             'A':{'C':1,'D':6},
             'B':{'E':1,'F2':9},
             'C':{'D':4,'I':10},
             'D':{'B':3,'F1':6},
             'F1':{'C':2},
             'E':{'F2':5},
             'F2':{'B':8}}

    hDuplo = {'A':5,'B':5,'C':3,'D':7,'E':4,'F1':0,'F2':0, 'I':10 }

    
    hGun = {'A':8,'B':9,'C':10,'D':6,'E':9,'F':8,'G':5,'H':5,'I':3,'J':0,'K':0,'L':1,'M':1 }

    def __init__(self,initial = 'A', goal = 'K',grafo=grafoDuplo,h=hDuplo) :
        super().__init__(initial,goal)
        self.grafo=grafo
        self.h=h
        
    def actions(self,estado) :
        sucessores = self.grafo[estado].keys()  # métodos keys() devolve a lista das chaves do dicionário
        accoes = list(map(lambda x : "ir de {} para {}".format(estado,x),sucessores))
        return accoes

    def result(self, estado, accao) :
        """Assume-se que uma acção é da forma 'ir de X para Y'
        """
        return accao.split()[-1]
    
    def path_cost(self, c, state1, action, state2):
        return c + self.grafo[state1][state2]
    
    def display(self,state):
        return state

    def h1(self,no):
        return self.h[no.state]
```

## Exemplos

### Exemplo 1
Primeiro teste com o problema do grafo duplo na imagem em cima

Vamos testar o IDA* em grafo com contagem do número de visitados

```python
s = ProblemaGrafo()
print('---------------- IDA* ----------------')
f=lambda n: n.path_cost + s.h1(n)
res_IDAstar,visitados=ida_star_graph_search_count(s,f)
if res_IDAstar:
    print("\nSolução:",res_IDAstar.solution(),'com custo',res_IDAstar.path_cost)
else:
    print('\nSem Solução')
print('Visitados:',visitados)
``` 

```python
---------------- IDA* ----------------

Solução: ['ir de I para B', 'ir de B para F2'] com custo 12
Visitados: 14
```

Agora com toque pedagógico:

```python
s = ProblemaGrafo()
print('---------------- IDA* pedagógico ----------------')
f=lambda n: n.path_cost + s.h1(n)
res_IDAstar,visitados=ida_star_graph_search_count(s,f,True)
if res_IDAstar:
    print("\nSolução:",res_IDAstar.solution(),'com custo',res_IDAstar.path_cost)
else:
    print('\nSem Solução')
print('Visitados:',visitados)
```

```python
---------------- IDA* pedagógico ----------------
------Cutoff at 10

I
Cost: 0 f= 10

A
Cost: 3 f= 8

C
Cost: 4 f= 7

D
Cost: 9 f= 16
Out of cutoff -- minimum out: 16

B
Cost: 3 f= 8

E
Cost: 4 f= 8

F2
Cost: 12 f= 12
Out of cutoff -- minimum out: 12


------Cutoff at 12

I
Cost: 0 f= 10

A
Cost: 3 f= 8

C
Cost: 4 f= 7

D
Cost: 9 f= 16
Out of cutoff -- minimum out: 16

B
Cost: 3 f= 8

E
Cost: 4 f= 8

F2
Cost: 12 f= 12
Goal found within cutoff!

Solução: ['ir de I para B', 'ir de B para F2'] com custo 12
Visitados: 14
```

### Exemplo 2
Usemos agora uma variante do grafo duplo mas sem ciclos em que procuramos um estado final que não existe. O IDA* terá que falhar, devolvendo `None` + o número de visitados.

```python
grafoDuploLimpo= {'I':{'A':3,'B':3},
             'A':{'C':1,'D':6},
             'B':{'E':1,'F2':9},
             'C':{'D':4},
             'D':{'B':3,'F1':6},
             'F1':{'C':2},
             'E':{'F2':5},
             'F2':{}}

hDuploLimpo = {'A':5,'B':5,'C':3,'D':7,'E':4,'F1':0,'F2':0, 'I':10 }
    
s = ProblemaGrafo('I',['X'],grafoDuploLimpo,hDuploLimpo)
```

e chamemos a função pedida sem o modo pedagógico activo:

```python
print('---------------- IDA* ----------------')
f=lambda n: n.path_cost + s.h1(n)
res_IDAstar,visitados=ida_star_graph_search_count(s,f)
if res_IDAstar:
    print("\nSolução:",res_IDAstar.solution(),'com custo',res_IDAstar.path_cost)
else:
    print('\nSem Solução')
print('Visitados:',visitados)
```

```python
---------------- IDA* ----------------

Sem Solução
Visitados: 22
```

Agora com o modo `pédagogique active`.

```python
print('---------------- IDA* ----------------')
f=lambda n: n.path_cost + s.h1(n)
res_IDAstar,visitados=ida_star_graph_search_count(s,f,True)
if res_IDAstar:
    print("\nSolução:",res_IDAstar.solution(),'com custo',res_IDAstar.path_cost)
else:
    print('\nSem Solução')
print('Visitados:',visitados)
``` 

```python
---------------- IDA* ----------------
------Cutoff at 10

I
Cost: 0 f= 10

A
Cost: 3 f= 8

C
Cost: 4 f= 7

D
Cost: 9 f= 16
Out of cutoff -- minimum out: 16

B
Cost: 3 f= 8

E
Cost: 4 f= 8

F2
Cost: 12 f= 12
Out of cutoff -- minimum out: 12


------Cutoff at 12

I
Cost: 0 f= 10

A
Cost: 3 f= 8

C
Cost: 4 f= 7

D
Cost: 9 f= 16
Out of cutoff -- minimum out: 16

B
Cost: 3 f= 8

E
Cost: 4 f= 8

F2
Cost: 12 f= 12


------Cutoff at 16

I
Cost: 0 f= 10

A
Cost: 3 f= 8

C
Cost: 4 f= 7

D
Cost: 9 f= 16

F1
Cost: 15 f= 15

B
Cost: 3 f= 8

E
Cost: 4 f= 8

F2
Cost: 12 f= 12

Sem Solução
Visitados: 22
```

### Exemplo 3
Voltemos ao nosso problema preferido, o Medo Total

```python
parametros="T=15\nM=6\nP=10"
linha1= "= = = = = =\n"
linha2= "= * F @ * =\n"
linha3= "= . . . . =\n"
linha4= "= . = . . =\n"
linha5= "= . = . . =\n"
linha6= "= = = = = =\n"
grelha=linha1+linha2+linha3+linha4+linha5+linha6
mundoStandard2=parametros + "\n" + grelha
gx=MedoTotal(mundoStandard2)
print(gx.display(gx.initial))
```

``` python
Tempo: 15
Medo: 6
= = = = = = 
= * F @ * = 
= . . . . = 
= . = . . = 
= . = . . = 
= = = = = = 
```

Apliquemos a função que aplica o IDA*

``` python
print('---------------- IDA* ----------------')
f=lambda n: n.path_cost + gx.minimal_h(n)
res_IDAstar,visitados=ida_star_graph_search_count(gx,f)
if res_IDAstar:
    print("\nSolução:",res_IDAstar.solution(),'com custo',res_IDAstar.path_cost)
else:
    print('\nSem Solução')
print('Visitados:',visitados)
```

```python
---------------- IDA* ----------------

Solução: ['E', 'S', 'S', 'S', 'W', 'N', 'N', 'W', 'W', 'N', 'S', 'S', 'S', 'N', 'S'] com custo 18
Visitados: 1919
```

Talvez não seja boa ideia mostrarmos o rasto do processo de procura, porque poderá ser pesadamente pedagógico e o mesmo acontece com o exemplo seguinte.

### Exemplo 4
Vejemos um puzzle MedoTotal mais desafiante e comparemos o tempo para chegar à solução, com o A* nas duas versões grafo e em árvore.

```python
parametros="T=45\nM=6\nP=10"
linha1= "= = = = = = = = = =\n"
linha2= "= @ . * . . * . . =\n"
linha3= "= . = = = = = = . =\n"
linha4= "= . = F * . . . . =\n"
linha5= "= . = = = = = = . =\n"
linha6= "= . = . . . . . . =\n"
linha7= "= . = . . . . . . =\n"
linha8= "= * . . . . . . . =\n"
linha9= "= . . . . . . . . =\n"
linha10="= = = = = = = = = =\n"
grelha=linha1+linha2+linha3+linha4+linha5+linha6+linha7+linha8+linha9+linha10
mundoStandardx=parametros + "\n" + grelha
gx=MedoTotal(mundoStandardx)
print(gx.display(gx.initial))
``` 

```python
Tempo: 45
Medo: 6
= = = = = = = = = = 
= @ . * . . * . . = 
= . = = = = = = . = 
= . = F * . . . . = 
= . = = = = = = . = 
= . = . . . . . . = 
= . = . . . . . . = 
= * . . . . . . . = 
= . . . . . . . . = 
= = = = = = = = = =
```

O A* em árvore

```python
print('---------------- A* tree----------------')
start = timeit.default_timer()
res_astar,expandidos = astar_search_tree_count(gx,gx.minimal_h)
stop = timeit.default_timer()
timeGraph = stop-start
print('Time:', timeGraph)
print('Expandidos',expandidos)
if res_astar:
    print("Solução:",res_astar.solution(),'com custo',res_astar.path_cost)
else:
    print('Sem Solução')
```

``` python
---------------- A* tree----------------
Time: 1.29444239999998
Expandidos 35382
Solução: ['S', 'S', 'S', 'S', 'S', 'S', 'E', 'W', 'N', 'N', 'N', 'N', 'N', 'N', 'E', 'E', 'E', 'W', 'W', 'E', 'E', 'E', 'W', 'E', 'E', 'E', 'W', 'E', 'E', 'S', 'S', 'W', 'W', 'W', 'W', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'S', 'S', 'W'] com custo 66
```

O A* em grafo

```python
print('---------------- A* grafo----------------')
start = timeit.default_timer()
res_astar,expandidos = astar_search_plus_count(gx,gx.minimal_h)
stop = timeit.default_timer()
timeGraph = stop-start
print('Time:', timeGraph)
print('Expandidos',expandidos)
if res_astar:
    print("Solução:",res_astar.solution(),'com custo',res_astar.path_cost)
else:
    print('Sem Solução')
```

```python
---------------- A* grafo----------------
Time: 2.385914800000137
Expandidos 21974
Solução: ['S', 'S', 'S', 'S', 'S', 'S', 'E', 'W', 'N', 'N', 'N', 'N', 'N', 'N', 'E', 'E', 'W', 'E', 'E', 'W', 'E', 'E', 'W', 'E', 'E', 'E', 'W', 'E', 'E', 'S', 'S', 'W', 'W', 'W', 'W', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'S', 'S', 'W'] com custo 66
```

O IDA* em grafo

```python
print('---------------- IDA* grafo----------------')
start = timeit.default_timer()
f=lambda n: n.path_cost + gx.minimal_h(n)
res_IDAstar,visitados=ida_star_graph_search_count(gx,f)
stop = timeit.default_timer()
timeGraph = stop-start
print('Time:', timeGraph)
if type(res_astar)!=tuple:
    print("Solução:",res_IDAstar.solution(),'com custo',res_IDAstar.path_cost)
else:
    print('Sem Solução')
print('Visitados:',visitados)
```

```python
---------------- IDA* grafo----------------
Time: 5.241901999999982
Solução: ['S', 'S', 'S', 'S', 'S', 'S', 'E', 'W', 'N', 'N', 'N', 'N', 'N', 'N', 'E', 'E', 'W', 'W', 'E', 'E', 'E', 'E', 'W', 'E', 'E', 'E', 'W', 'E', 'E', 'S', 'S', 'W', 'W', 'W', 'W', 'E', 'E', 'E', 'E', 'S', 'S', 'W', 'W', 'W', 'W'] com custo 66
Visitados: 144987
```

Fica para voçês pensarem porque será que o A* em árvore foi melhor do que a versão em grafo. E será que isso acontece para mundos Medo Total mais complexos. Neste exemplo, o IDA* em grafo revelou-se mais lento do que o A*. E a versão IDA* em árvore seria também melhor do que a versão em grafo? E será mais competitiva do que o A* para mundos MedoTotal mais complexos e mais desafiantes?

## Cotação
Esta pergunta vale 0.6 valores num total de 1.75 da avaliação contínua.

<img src="Imagens\rollingAsteriskus.gif" alt="Drawing" style="width: 200px;"/>